# Identify the Field and all Obstacles Using a Map Generated by ROS

This script reads a map generated by ROS using the laser sensor, and identifies the field where the robot can move and the obstacles in the field.

In [1]:
import cv2
import numpy as np
import imutils
from PIL import Image
from collections import Counter

def show_img(image):
    im = Image.fromarray(image)
    im.show()

def show_pixels(image):
    dic = Counter(image.flatten())
    print(dic)

In [2]:
mapimg = cv2.imread("images/map.png")
mapimg.shape

(400, 400, 3)

<img src="images/map.png" width="30%"/>

In [3]:
# Apply threshold on the gray-scale image
gray = cv2.cvtColor(mapimg, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
thresh = cv2.threshold(blurred, 206, 255, cv2.THRESH_BINARY_INV)[1]
#show_img(thresh)
print(thresh.shape)

(400, 400)


In [4]:
# Remove pixels that add noise to the image
kernel = np.ones((5,5),np.uint8)
low_noise = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
#show_pixels(low_noise)
#show_img(low_noise)

In [5]:
# Invert the color of the image
low_noise_inv = np.invert(low_noise)
#show_img(low_noise_inv)
show_pixels(low_noise_inv)

Counter({0: 149032, 255: 10968})


In [6]:
# Find contours of the field and not the obstacles
cnts = cv2.findContours(low_noise_inv.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

max_area = 0.0
field = []
for cnt in cnts:
    area = cv2.contourArea(cnt)
    if area > max_area:
        field = cnt
        max_area = area

rect = cv2.minAreaRect(field)
box = cv2.boxPoints(rect)
box = np.int0(box)
cv2.drawContours(mapimg, [box], -1, (255,0,0), 2)
cv2.imwrite('images/field.png', mapimg)
print("Image saved as: images/field.png")

Image saved as: images/field.png


<img src="images/field.png" width="30%"/>

In [7]:
# Create a mask of the field by filling the polygon
mask = np.zeros(low_noise_inv.shape, dtype=np.uint8)
cv2.fillPoly(mask, pts =[box], color=(255,255,255))
#show_img(mask)
show_pixels(mask)

Counter({0: 149240, 255: 10760})


In [8]:
# Apply the mask on the obstacles to keep only the working content
field_obst =  cv2.bitwise_and(low_noise_inv, low_noise_inv, mask= mask)
#show_img(field_obst)
show_pixels(field_obst)

Counter({0: 149983, 255: 10017})


In [9]:
# Invert pixels of the field_obst image
field_obst_inv = np.invert(field_obst)
#show_img(field_obst_inv)
show_pixels(field_obst_inv)

Counter({255: 149983, 0: 10017})


In [10]:
# Create an inverted mask
mask_inv = np.invert(mask)
#show_img(mask_inv)
show_pixels(mask_inv)

Counter({255: 149240, 0: 10760})


In [11]:
# Identify only the obstacles in the image
tmp = field_obst_inv - mask_inv
obstacles = cv2.morphologyEx(tmp, cv2.MORPH_OPEN, kernel)
#cv2.drawContours(obstacles, [box], -1, (0,0,0), 2)
#show_img(obstacles)
show_pixels(obstacles)

Counter({0: 159611, 255: 389})


In [12]:
# Indentify the center of each obstacle and draw a circle in each center
cnts_obj = cv2.findContours(obstacles.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts_obj = imutils.grab_contours(cnts_obj)

for cnt in cnts_obj:
    (x, y), radius = cv2.minEnclosingCircle(cnt)
    center = (int(x), int(y))
    radius = int(radius)
    cv2.circle(mapimg, center, radius, (0, 0, 255), 2)

    #rect = cv2.minAreaRect(cnt)
    #box = cv2.boxPoints(rect)
    #box = np.int0(box)
    #cv2.drawContours(mapimg, [box], -1, (255,0,0), 2)
cv2.imwrite('images/obstacles.png', mapimg)
print('Image saved as: images/obstacles.png')

Image saved as: images/obstacles.png


<img src="images/obstacles.png" width="30%"/>